# Save calibrated hydraulic conductivity
Reads hidraulic conductvities from a npf file saved by PEST. The saved file could be imported in ModelMuse<br>
Ahora solo exporta una capa a la vez

In [28]:
import flopy.mf6 as mf6
import matplotlib.pyplot as plt
from pathlib import Path

In [29]:
def centroid_get(vertex_xy:[[]]):
    cx = 0.
    cy = 0.
    n = len(vertex_xy)
    for coordinate in vertex_xy:
        cx += coordinate[0]
        cy += coordinate[1]
    return cx/n, cy/n

def plot_shape(vertex_xy, point1):
    """
    Plot the shape defined by the vertex_xy list and add the point1 to the shape.

    Parameters:
    vertex_xy (list): List of coordinates that define the shape.
    point1 (tuple): Coordinate of the point to be added to the shape.

    Returns:
    Figure object containing the plot.
    """
    fig, ax = plt.subplots()

    x = []
    y = []

    for x_coord, y_coord in vertex_xy:
        x.append(x_coord)
        y.append(y_coord)

    ax.plot(x, y, color='blue')
    ax.scatter(x, y, marker='o', color='blue')

    # Add point1 to the plot
    ax.plot([point1[0]], [point1[1]], marker='o', color='red')

    # Set aspect ratio and labels
    ax.set_aspect('equal')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Shape Plot')

    plt.show()


### Set variables

In [12]:
# model 
modelname = 'sc.nam'
model_path = r'E:\LSGB\20231117_JCUNVRS\gwf\gwf3_pest1'
model_path = Path(model_path)
epsg_code = 25830

# layer
layer = 0

# dir path to save k files
dir_path = r'E:\LSGB\20231117_JCUNVRS\gwf\calibrations\20240127a' 
dir_path = Path(dir_path)
kx_name = 'kx.csv' 
ky_name = 'ky.csv'
kz_name = 'kz.csv'

# exports only active cells 
exp_only_grid_active = True

# header
write_header = True

# ('point', 'ewkt_polygon')
geometry = 'point'

### Read for each simulation

In [7]:
# my model
sim = mf6.MFSimulation.load(sim_name=modelname, exe_name='mf6', sim_ws=model_path)

model_names = list(sim.model_names)
print('\nModel names in sim', model_names)

gwf = sim.get_model(model_names[0])

# general information of my model
package_list = gwf.get_package_list()
print('Packages:', package_list)
print('Number of layers:', gwf.dis.nlay.array)  # Number of layers
print('Number of cells:', gwf.dis.ncpl.array)  # Number of cells per layer
print('Number of active cells:',(gwf.dis.idomain.array > 0).sum())

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package ic...
    loading package npf...
    loading package obs...
    loading package gnc...
    loading package oc...
    loading package drn...
    loading package rch...
  loading solution package modflow...

Model names in sim ['modflow']
Packages: ['DISV', 'IC', 'NPF', 'OBS-1', 'GNC', 'OC', 'DRN-1', 'RCH-1']
Number of layers: 1
Number of cells: 2331
Number of active cells: 1606


### Lee y escribe los ficheros de conductividad hidráulica

In [13]:
# Get the packages
disv = gwf.get_package('disv')
npf = gwf.get_package('npf')

# idomain
idomain = disv.idomain.array

# cells
cell2d = disv.cell2d.array

# vertices
vertices = disv.vertices.array

# hydraulic conductivities
k11 = npf.k.array
k22 = npf.k22.array
k33 = npf.k33.array

header = 'layer,cell,idomain,x,y,k'
kx_file = dir_path.joinpath(kx_name)
ky_file = dir_path.joinpath(ky_name)
kz_file = dir_path.joinpath(kz_name)

with open(kx_file, 'w', encoding='utf8') as fkx, \
    open(ky_file, 'w', encoding='utf8') as fky, \
    open(kz_file, 'w', encoding='utf8') as fkz:
    if write_header:
        fkx.write(f'{header}x\n')
        fky.write(f'{header}y')
        fkz.write(f'{header}z\n')
    for ic, cell in enumerate(cell2d):
        lcell = list(cell)
        # Number of vertices for the cell
        num_vertices = lcell[3]
        # Extract the vertex identifiers
        vertex_ids = lcell[4:4 + num_vertices]
        vertex_xy = [list(vertices[v1])[1:] for v1 in vertex_ids]
        centroid = centroid_get(vertex_xy)
        # Format coordinates as a string
        coordinate_str = ', '.join([f'{x} {y}' for x, y in vertex_xy])
        # Create EWKT representation
        ewkt_representation = f'SRID={epsg_code};POLYGON(({coordinate_str}))'
        if exp_only_grid_active:
            if idomain[layer,lcell[0]] == 1:
                write_cell= True
            else:
                write_cell = False
        else:
            write_cell = True
        if write_cell:
            if geometry == 'point':
                fkx.write(f'{layer},{lcell[0]},{idomain[layer,lcell[0]]},{centroid[0]:f},{centroid[1]:f},{k11[layer,lcell[0]]:f}\n')
                fky.write(f'{layer},{lcell[0]},{idomain[layer,lcell[0]]},{centroid[0]:f},{centroid[1]:f},{k22[layer,lcell[0]]:f}\n')
                fkz.write(f'{layer},{lcell[0]},{idomain[layer,lcell[0]]},{centroid[0]:f},{centroid[1]:f},{k33[layer,lcell[0]]:f}\n')
            else:
                fkx.write(f'{layer},{lcell[0]},{idomain[layer,lcell[0]]},{centroid[0]:f},{centroid[1]:f},{k11[layer,lcell[0]]:f}\n')
                fky.write(f'{layer},{lcell[0]},{idomain[layer,lcell[0]]},{centroid[0]:f},{centroid[1]:f},{k22[layer,lcell[0]]:f}\n')
                fkz.write(f'{layer},{lcell[0]},{idomain[layer,lcell[0]]},{centroid[0]:f},{centroid[1]:f},{k33[layer,lcell[0]]:f}\n')


First cell: [0, 608467.3350144, 4257604.522862, 4, 0, 1, 40, 39, None, None]
